In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import muspy
from pathlib import Path
from tqdm.notebook import tqdm
from src.utils import sequence_extraction

# Sequence Extraction with a Single Song

In [21]:
# song = muspy.read('../midi/examples/bitmidi/pink-panther.mid')
song = muspy.read('../data/reference_data/raw/theorytab_midi/a/abba/one-of-us/verse_symbol_key.mid')
print(song)
song[0][-1]

Music(metadata=Metadata(schema_version='0.0', source_filename='verse_symbol_key.mid', source_format='midi'), resolution=480, tempos=[Tempo(time=0, qpm=78.9999657666815)], key_signatures=[KeySignature(time=0, root=7, mode='major')], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], tracks=[Track(program=0, is_drum=False, name='melody', notes=[Note(time=240, pitch=67, duration=240, velocity=80), Note(time=480, pitch=67, duration=240, velocity=80), Note(time=720, pitch=66, duration=120, velocity=80), ...])])


Note(time=20040, pitch=66, duration=1080, velocity=80)

In [22]:
success = sequence_extraction.extract_melodies_to_folder(song, 'one-of-us', 'test_set', 4)
print("Sequence extraction returned", success)

Sequence extraction returned 1


In [23]:
# check if track is long enough for at least one call-and-response pair

end_time = song.tracks[0][-1].time / song.resolution
length_in_bars = 4
quarters_per_bar = song.time_signatures[0].numerator * 4 / song.time_signatures[0].denominator

if round(end_time / quarters_per_bar) >= length_in_bars * 2:
    print("Track is long enough.")

print("End time: %f" % end_time)
print("Duration in bars: " + str(end_time / quarters_per_bar))
print("Required duration in bars: " + str(length_in_bars*2))

End time: 0.015427
Duration in bars: 0.0038568054687500002
Required duration in bars: 8


# Sequence Extraction from a Dataset
Using a Set of crawled Midis from Theorytab

In [2]:
source_folder = '../data/reference_data/raw/theorytab_midi' 
sequence_extraction.split_all_midis_from_folder(source_folder, 'theorytab', 4)

0it [00:00, ?it/s]

18167 songs processed in728.5443201065063
1666 songs had no melody track.
4758 songs were too short.
0songs could not be processed due to an error.


In [13]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

def remove_suffix(text, suffix):
    if text.endswith(suffix):
        return text[:-len(suffix)]
    return text

In [14]:
source_folder = '../data/reference_data/raw/theorytab_midi'

i = 0
no = 0
er = 0

for root, dirs, files in tqdm(os.walk(source_folder)):
    for file in files:
        print("Processing " + root, end="\r") 
        if file.endswith(".mid"):
            try:
                song = muspy.read(os.path.join(root, file))
                file_name = remove_prefix(root, source_folder + "\\").replace("\\", "_") + "_" + remove_suffix(file, "_symbol_key.mid")
                success = sequence_extraction.extract_melodies_to_folder(song, file_name, 'theorytab2', 4)
                if (success == 0):  
                    print("No melody track found for " + file_name)
                    no += 1
                i += 1
            except ValueError as error:
                print("Error while processing " + file_name + ": " + error)

print(i, no, er)

0it [00:00, ?it/s]

y_intro
No melody track found for v_virna-lindt_wild-strawberries_verse
No melody track found for v_virtual-riot_energy-drink_chorus
No melody track found for v_virtual-self_ghost-voices_verse
No melody track found for v_virtual-self_key_chorus
No melody track found for v_vitas_7th-element_chorus-lead-out
No melody track found for v_vitas_7th-element_chorus
No melody track found for v_vitas_7th-element_verse
No melody track found for v_vitas_opera-2_chorus
No melody track found for v_vlad_magicka-2_verse-and-pre-chorus
No melody track found for v_vulfpeck_game-winner_chorus
No melody track found for v_vulfpeck_game-winner_verse
No melody track found for v_vulfpeck_my-first-car_verse
No melody track found for v_vulfpeck_outro_verse
No melody track found for v_vulfpeck_wait-for-the-moment_verse
No melody track found for w_waka-flocka-flame_no-hands_chorus
No melody track found for w_walk-off-the-earth_lightning-bolt_verse
No melody track found for w_walk-the-moon_shut-up-and-dance_intro-